In [4]:
torch.__version__

'1.10.0+cu102'

In [1]:
import warnings
warnings.filterwarnings(action='ignore')

import os
import pandas as pd
import numpy as np

from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split
import IPython.display as ipd
import time

In [3]:
import torch
import torchvision
from torch import Tensor, nn, optim
from torch.utils.data import Dataset, DataLoader

import torch.nn.functional as F
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.models as models
from adamp import AdamP
from torchsummary import summary

import albumentations as A
from albumentations.pytorch import ToTensor
from efficientnet_pytorch import EfficientNet

## 시드 고정

In [5]:
import random
def seed_everything(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)  
    torch.backends.cudnn.deterministic = True  
    torch.backends.cudnn.benchmark = True  
seed_everything()

## 데이터 불러오기

In [6]:
train = pd.read_csv('./train/train_data.csv')
test = pd.read_csv('./test/test/test_data.csv')

In [7]:
train.columns = ['file_name','label']

In [23]:
train.head()

,file_name,label
0,train0001.png,8
1,train0002.png,8
2,train0003.png,8
3,train0004.png,8
4,train0005.png,8


In [8]:
test.head()

,file_name
0,idx0001.png
1,idx0002.png
2,idx0003.png
3,idx0004.png
4,idx0005.png


In [9]:
class CustomDataset(torch.utils.data.Dataset): 
    def __init__(self,df,path,option,augmentation=None):
        self.df = df
        self.option = option
        self.augmentation = augmentation
        self.path = path

    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx): 
        file_path  = self.df.iloc[idx,0]
        image=  Image.open(os.path.join(self.path,file_path)).convert('RGB')
        #image = image.resize((100,100))
        image = np.array(image)
        
        #image = transforms.ToTensor()(image=image)['image']
        
        
        if self.augmentation is not None:
            image = self.augmentation(image=image)['image']
        
        
        if self.option =='train':
            label = self.df.iloc[idx,1]
            label = torch.tensor(label, dtype=torch.int64)
            return image, label
        
        return image
    

In [10]:
class CNN(torch.nn.Module):

    def __init__(self):
        super(CNN, self).__init__()
        # 첫번째층
        # ImgIn shape=(batch_size, 28, 28, 2)
        #    Conv     -> (batch_size, 28, 28, 16)
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(3, 16, kernel_size=3, stride=1, padding='same'),
            torch.nn.ReLU(),
            torch.nn.BatchNorm2d(16),
            torch.nn.Dropout(p=0.3))
        
        # 두번째층
        # ImgIn shape=(batch_size, 28, 28, 16)
        #    Conv      ->(batch_size, 28, 28, 32)
        #    Pool      ->(batch_size, 9, 9, 32)
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(16, 32, kernel_size=3, stride=1, padding='same'),
            torch.nn.ReLU(),
            torch.nn.BatchNorm2d(32),
            torch.nn.Conv2d(32, 32, kernel_size=5, stride=1, padding='same'),
            torch.nn.ReLU(),
            torch.nn.BatchNorm2d(32),
            torch.nn.Conv2d(32, 32, kernel_size=5, stride=1, padding='same'),
            torch.nn.ReLU(),
            torch.nn.BatchNorm2d(32),
            torch.nn.Conv2d(32, 32, kernel_size=5, stride=1, padding='same'),
            torch.nn.ReLU(),
            torch.nn.BatchNorm2d(32),
            torch.nn.MaxPool2d(kernel_size=3, stride=3),
            torch.nn.Dropout(p=0.3))
        
        # 세번째층
        # ImgIn shape=(batch_size, 9, 9, 32)
        #    Conv      ->(batch_size, 9, 9, 64)
        #    Pool      ->(batch_size, 3, 3, 64)        
        self.layer3 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding='same'),
            torch.nn.ReLU(),
            torch.nn.BatchNorm2d(64),
            torch.nn.Conv2d(64, 64, kernel_size=3, stride=1, padding='same'),
            torch.nn.ReLU(),
            torch.nn.BatchNorm2d(64),
            torch.nn.MaxPool2d(kernel_size=3, stride=3),
            torch.nn.Dropout(p=0.3))
        
        # 전결합층 7x7x64 inputs -> 10 outputs
        self.fc = torch.nn.Linear(3*3*64, 10, bias=True)

        torch.nn.init.xavier_uniform_(self.fc.weight) # fc 가중치 초기화

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0), -1)   # Flatten
        out = self.fc(out)
        return out

In [11]:
class MnistModel(nn.Module):
    def __init__(self,):
        super().__init__()
        #self.conv2d = nn.Conv2d(3, 3, 3, stride=1)
        self.model = torchvision.models.resnet18(pretrained = False)
        #self.model = EfficientNet.from_name('efficientnet-b1')
        self.dropout = nn.Dropout(0.5)
        
        self.classifier = nn.Linear(1000, 10)

    def forward(self, images):
        #outputs = self.conv2d(images)
        outputs = self.model(images)
        outputs = self.dropout(outputs)
        outputs = self.classifier(outputs)
        return outputs

In [12]:
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits = 4)
folds=[]
for trn_idx,val_idx in skf.split(train['file_name'],train['label']):
    folds.append((trn_idx,val_idx))

In [13]:
num_epochs = 30
best_models = [] # 폴드별로 가장 validation acc가 높은 모델 저장

for i,fold in enumerate(range(4)):
    print('===============',i+1,'fold start===============')
    
    model = CNN().to('cuda')
    optimizer = AdamP(model.parameters(), lr=1e-3, )
    criterion = nn.CrossEntropyLoss()
    lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                                step_size = 5,
                                                gamma = 0.9)
        
    
    train_idx = folds[fold][0]
    valid_idx = folds[fold][1]
    
    train_data = train.loc[trn_idx]
    val_data = train.loc[valid_idx]
    
    train_transforms = A.Compose([
    A.Normalize(),
    A.HorizontalFlip(),
    A.RandomRotate90(p=0.5),
    A.VerticalFlip(p=0.5),
    ToTensor()
    ])

    test_transforms = A.Compose([
        A.Normalize(),
        ToTensor()
        ])

    train_dataset = CustomDataset(train_data,'train/','train',train_transforms)
    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=0)

    valid_dataset = CustomDataset(val_data,'train/','train',test_transforms)
    valid_loader = DataLoader(valid_dataset, batch_size=64, shuffle=False, num_workers=0)
    
    
    valid_acc_max = 0.85
    valid_loss_min = 0.4
    for epoch in range(num_epochs):

        for i, (images, targets) in enumerate(train_loader):
            model.train()
            images, targets = images.to('cuda'), targets.to('cuda')

            optimizer.zero_grad()
            outputs = model(images)

            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()  

            if (i+1) % 20 == 0:
                print(f'Epoch: {epoch} - Loss: {loss:.6f}')


        val_loss = []
        val_acc = []
        for i, (images, targets) in enumerate(valid_loader):
            model.eval()
            images, targets = images.to('cuda'), targets.to('cuda')

            with torch.no_grad():
                outputs = model(images)
                valid_loss = criterion(outputs,targets).cpu().detach().numpy()

                preds = torch.argmax(outputs,axis = 1)
                preds = preds.cpu().detach().numpy()

                targets = targets.cpu().detach().numpy()
                batch_acc = (preds==targets).mean()



                val_loss.append(valid_loss)
                val_acc.append(batch_acc)

        val_loss = np.mean(val_loss)
        val_acc = np.mean(val_acc)

        print(f'Epoch: {epoch} - valid Loss: {val_loss:.6f} - valid_acc : {val_acc:.6f}')
        
        '''
         if valid_acc_max < val_acc:
            valid_acc_max = val_acc
            best_models.append(model)
            print('model save, model val acc : ',val_acc)
            print('best_models size : ',len(best_models))
            '''


        if valid_loss_min > val_loss:
            valid_loss_min = val_loss
            best_models.append(model)
            
    # Learning rate 조절
    lr_scheduler.step()
    
  
    
        
        

=============== 1 fold start===============
Epoch: 0 - Loss: 3.040205
Epoch: 0 - Loss: 2.497284
Epoch: 0 - valid Loss: 1.188079 - valid_acc : 0.532858
Epoch: 1 - Loss: 1.771061
Epoch: 1 - Loss: 1.161166
Epoch: 1 - valid Loss: 1.350177 - valid_acc : 0.525230
Epoch: 2 - Loss: 1.223000
Epoch: 2 - Loss: 1.093718
Epoch: 2 - valid Loss: 0.548030 - valid_acc : 0.824770
Epoch: 3 - Loss: 1.484946
Epoch: 3 - Loss: 0.914585
Epoch: 3 - valid Loss: 0.683577 - valid_acc : 0.780515
Epoch: 4 - Loss: 1.182467
Epoch: 4 - Loss: 0.908000
Epoch: 4 - valid Loss: 0.370368 - valid_acc : 0.886719
Epoch: 5 - Loss: 0.756857
Epoch: 5 - Loss: 0.797708
Epoch: 5 - valid Loss: 0.354504 - valid_acc : 0.887500
Epoch: 6 - Loss: 0.754496
Epoch: 6 - Loss: 0.639200
Epoch: 6 - valid Loss: 0.323372 - valid_acc : 0.893153
Epoch: 7 - Loss: 0.784084
Epoch: 7 - Loss: 0.394101
Epoch: 7 - valid Loss: 0.192939 - valid_acc : 0.939844
Epoch: 8 - Loss: 0.468422
Epoch: 8 - Loss: 0.294915
Epoch: 8 - valid Loss: 0.242287 - valid_acc : 0.

In [16]:
test_transforms = A.Compose([
    A.Normalize(),
    ToTensor()
])

test_dataset = CustomDataset(test,'test/test','test',test_transforms)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=0)

In [17]:
len(best_models)

39

In [18]:
preds = []
for idx,model in enumerate(best_models[:1]): 
    print(idx+1, '번째 모델 예측 진행중')
    model = model
    model.eval()
    y_pred = []
    with torch.no_grad():
        for i, d in enumerate(test_loader):
            inputs = d.to('cuda')
            outputs = model(inputs).detach().cpu().numpy()
            y_pred.extend(outputs.argmax(axis=1).astype(int))
            
    preds.append(y_pred)




1 번째 모델 예측 진행중


In [19]:
from collections import Counter
np_pred = np.array(preds).T

pred = []
for i in range(5000):
    cnt = Counter(np_pred[i])
    pred.append(cnt.most_common()[0][0])
#pred

In [20]:
submission = pd.read_csv('sample_submission.csv')
submission['label'] = pred

In [21]:
submission.to_csv('sub_211124.csv',index = False)

In [22]:
submission['label'].value_counts()

7    649
6    639
1    546
4    529
8    517
9    489
0    467
2    459
3    356
5    349
Name: label, dtype: int64